In [1]:
import paraview.simple as pv
import numpy as np

Small script to calculate the area-averaged WSS for aneurysm case with different meshes for mesh convergence study.
The area-averaged WSS is defined as:

\begin{equation}
    WSS_A 
    =
    \frac{1}{A}
    \int_{S}{}
        \left\| \vec{WSS} (\vec{x},t) \right\| 
        \mathrm{d}S   
\end{equation}

where A is the area of the surface. Note that WSS_A will be a function of time. According to authors like Roach and Ferziger, in a Richardson extrapolation-like analysis, it is possible to use integral or derivatives of the priumitive variables to analyze mesh convergence. Ideally, one should analyze both primitive variables in specific locations and quadratures of these that are the variables that will be used in the study being carried-out. First, we will integrate the variables on the chosen surface and in the sequence, we must multiply the magWSS resultant field by the density $\rho = 1056 kg/m^3$, because the loaded 'wallShearComponent' in OpenFOAM is calculated per specific mass unity. 

In [63]:
def area_averaged_wss(case):
    """ Function that calculates the area-averaged WSS
        for a surface where the wall shear stress field
        is defined. The function takes its input an 
        OpenFOAM case reader with the surface and fields.
        It automatically selects the wallShearComponent field
        and the surface wall. It returns an array with time
        in one column and the result in the other. 
    """
    # Update arrays to be used:
    # - wallShearComponent
    

    case.CellArrays = ['wallShearComponent']

    # And select the surface where integration will be carried out
    case.MeshRegions = ['wall']

    # Get time-steps values
    timeSteps = np.array(case.TimestepValues)
    # Update time-step
    case.UpdatePipeline()

    # Integrate WSS on the wall
    computeWallArea = pv.IntegrateVariables()
    computeWallArea.Input = case
    computeWallArea.UpdatePipeline()
    # Get area of surface, in m2
    wallArea = computeWallArea.CellData.GetArray('Area').GetRange()[0]

    areaAveragedWSSList = []
    for timeStep in timeSteps:
        # Calculate WSS magnitude
        # Instantiate calculater filter
        calcMagWSS = pv.Calculator()
        calcMagWSS.Input = case
        calcMagWSS.ResultArrayName = 'magWSS'
        calcMagWSS.Function = '1056*mag(wallShearComponent)'
        calcMagWSS.AttributeType = 'Cell Data'
        calcMagWSS.UpdatePipeline(time=timeStep)

        # Integrate WSS on the wall
        integrateWSS = pv.IntegrateVariables()
        integrateWSS.Input = calcMagWSS
        integrateWSS.UpdatePipeline(time=timeStep)

        # Instantiate calculater filter
        areaAveragedWSS = pv.Calculator()
        areaAveragedWSS.Input = integrateWSS
        areaAveragedWSS.ResultArrayName = 'areaAveragedWSS'
        areaAveragedWSS.Function = 'magWSS/'+str(wallArea)
        areaAveragedWSS.AttributeType = 'Cell Data'
        areaAveragedWSS.UpdatePipeline(time=timeStep)
        areaAveragedWSSList.append([timeStep, 
                                    areaAveragedWSS.CellData.GetArray('areaAveragedWSS').GetRange()[0]])

    return np.asarray(areaAveragedWSSList)

In [59]:
pathToFoamCase = "/home/iago/foam/iago-4.0/run/aneurysms/unruptured/fluidFlow/Newtonian/case11/mesh600k/" 
foamFileName = "mesh600k.foam"

In [66]:
data = pv.OpenFOAMReader(FileName=pathToFoamCase+foamFileName)

In [67]:
data.CellData.GetNumberOfArrays()

5

In [68]:
meshIntermediateWSS = area_averaged_wss(data)

AttributeError: 'NoneType' object has no attribute 'GetRange'

In [57]:
meshIntermediateWSS

array([[ 0.1       , 11.96694554]])

In [56]:
import matplotlib.pyplot as plt
import seaborn as sb

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
plt.style.use('classic')

In [ ]:
#%matplotlib widget
fig = plt.figure()

plt.plot(meshIntermediateWSS[:,0], meshIntermediateWSS[:,1], 'b')

plt.xlabel('Time')
plt.ylabel('Area-averaged WSS (Pa)')

plt.grid()
plt.show()

In this part of this notebook we will created a sequence of commands to calculate the *oscillatory shear index* (OSI) of a WSS field in an aneurysm surface, from CFD results and using ParaView's Temporal Statistics filter. The function also computes time-averaged data of the WSS, necessary in the OSI definition, and also the *relative residence time (RRT)*.

The OSI is defined as:

\begin{equation}
    OSI(\vec{x})
    =
    \frac{1}{2}
    \left(
        1 
        -
        \frac{
            \left \| 
            \displaystyle\int_0^{T}
            {
                \overrightarrow{WSS} (\vec{x},t)
                \mathrm{d}t
            }
            \right \|
        }{
            \displaystyle\int_0^{T}
            {
                \left \| \vec{WSS} (\vec{x},t) \right \|
                \mathrm{d}t
            }
        }
    \right)
\end{equation}

it measures how the WSS direction changes in time. The RRT is defined as:

\begin{equation}
    RRT(\vec{x})
    =
    \frac{1}{
            \frac{1}{T}
                \left \| 
                    \displaystyle\int_0^{T}
                    {
                        \vec{WSS} (\vec{x},t)
                        \mathrm{d}t
                    }
                \right \|
      }
     =
     \frac{1}{(1 - 2OSI)\overline{WSS}}
\end{equation}


In [69]:
def osi(ofCaseFile, 
        timeIndexRange, 
        outputFileName,
        timeStep=0.01,
        density=1056.0, # kg/m3
        wssFieldName='wallShearComponent', 
        patchName='wall'):
    """ 
        Function to calculate the oscillatory shear index and
        other time integrals variables of WSS over a time inter-
        val [Ti,Tf] indentified by time-step indices. The method
        (based o VTK), ignores the time-step size and 
        consider uniform time stepping (so, if the time-step is 
        large, the resulting fields may be very different if a va-
        riable time-step would be considered). The OSI field is 
        defined as:
        
            OSI = 0.5*( 1 - norm2(int WSS dt) / int norm2(WSS) dt)
            
        where "int" implies the integral over time between two
        instants t1 and t2 (usually for a cardiac cycle, therefore 
        [t1, t2] = [Ti, Tf]) and norm2 is the L2 norm of an Eucli-
        dean vector field; WSS is the wall shear stress defined on 
        the input surface. Since this function use OpenFOAM data, 
        please specify the density considered.
        
        Input args:
        - OpenFOAM case file (str): name of OpenFOAM .foam case;
        - wssFieldName (str, optional): string containing the name 
            of the wall shear stress field (default="wallShearComp-
            onent");
        - patchName (str, optional): patch name where to calculate 
            the OSI (default="wall");
        - timeIndexRange (list): list of initial and final time-
            steps indices limits of the integral [Ti, Tf];
        - outputFileName (str): file name for the output file with 
            osi field (must be a .vtp file).
        - blood density (float, optional): default 1056.0 kg/m3
    """
    case = pv.OpenFOAMReader(FileName=ofCaseFile)

    # First we define only the field that are going to be used: the WSS on the aneurysm wall
    case.CellArrays = [wssFieldName]
    case.MeshRegions = [patchName]
    case.Createcelltopointfiltereddata = 0

    # Multiplying WSS per density
    densityTimesWSS = pv.Calculator()
    densityTimesWSS.Input = case
    densityTimesWSS.AttributeType = 'Cell Data'
    densityTimesWSS.ResultArrayName = "WSS"
    densityTimesWSS.Function = str(density)+"*"+wssFieldName
    densityTimesWSS.UpdatePipeline()

    # Calculating the magnitude of the wss vector
    calcMagWSS = pv.Calculator()
    calcMagWSS.Input = densityTimesWSS
    calcMagWSS.AttributeType = 'Cell Data'
    calcMagWSS.ResultArrayName = densityTimesWSS.ResultArrayName+"_magnitude"

    ## Get WSS field name
    wss = densityTimesWSS.ResultArrayName
    calcMagWSS.Function = "mag("+wss+")"
    calcMagWSS.UpdatePipeline()

    # Extract desired time range
    timeInterval = pv.ExtractTimeSteps()
    timeInterval.Input = calcMagWSS
    timeInterval.SelectionMode = "Select Time Range"
    timeInterval.TimeStepRange = timeIndexRange #[99, 199] # range in index
    timeInterval.UpdatePipeline()

    # Period given by time-steps
    period = (timeInterval.TimeStepRange[1] - timeInterval.TimeStepRange[0])*timeStep

    # Now compute the temporal statistics
    # filter computes the average values of all fields
    calcAvgWSS = pv.TemporalStatistics()

    calcAvgWSS.Input = timeInterval
    calcAvgWSS.ComputeAverage = 1
    calcAvgWSS.ComputeMinimum = 0
    calcAvgWSS.ComputeMaximum = 0
    calcAvgWSS.ComputeStandardDeviation = 0
    calcAvgWSS.UpdatePipeline()

    # Calculates OSI
    calcOSI = pv.Calculator()
    calcOSI.Input = calcAvgWSS
    calcOSI.ResultArrayName = 'OSI'
    calcOSI.AttributeType = 'Cell Data'

    # Getting fields:
    # - Get the average of the vector WSS field
    avgVecWSS = calcAvgWSS.CellData.GetArray(wss+"_average").GetName()
    # - Get the average of the magnitude of the WSS field 
    avgMagWSS = calcAvgWSS.CellData.GetArray(calcMagWSS.ResultArrayName+"_average").GetName()

    calcOSI.Function = "0.5*( 1 - ( mag( "+avgVecWSS+" ) )/"+avgMagWSS+" )"
    calcOSI.UpdatePipeline()

    # Compute Relative Residance Time
    calcRRT = pv.Calculator()
    calcRRT.Input = calcOSI
    calcRRT.ResultArrayName = 'RRT'
    calcRRT.AttributeType = 'Cell Data'
    calcRRT.Function = str(period)+"/mag("+avgVecWSS+")"
    calcRRT.UpdatePipeline()

    # Final processing of surface: merge blocks
    # and get surface for triangulation
    mergeBlocks = pv.MergeBlocks()
    mergeBlocks.Input = calcRRT
    mergeBlocks.UpdatePipeline()

    extractSurface = pv.ExtractSurface()
    extractSurface.Input = mergeBlocks
    extractSurface.UpdatePipeline()

    triangulate = pv.Triangulate()
    triangulate.Input = extractSurface
    triangulate.UpdatePipeline()

    pv.SaveData(outputFileName,triangulate)

In [70]:
pathToFoamCase = "/home/iago/foam/iago-4.0/run/aneurysms/unruptured/fluidFlow/Newtonian/case11/mesh600k/" 
foamFileName = "mesh600k.foam"

In [71]:
osi(pathToFoamCase+foamFileName,[0, 50],pathToFoamCase+"OSI_new.vtp")

# Computing integrals over space (aneurysm surface) of time-averaged WSS

The following code computes the integrals over the aneurysm surface of the time-averaged WSS, already calculated in a single surface by, for example, using the OSI function, above. Therefore, the code only clips the aneurysm neck in the same points as used above to compute the time-dependent WSS surface integrals. 

The procedure then will provide a representative value of WSS used for comparison with other publications. With the time-averaged defined as:

\begin{equation}
    \overline{WSS}(\vec{x})
    =
    \frac{1}{T}
    \int_0^{T}
    {
        \left \| \vec{WSS} (\vec{x},t) \right \|
        \mathrm{d}t
    }
\end{equation}

which is called "wallShearComponent_magnitude_average". This is, therefore, a single spatial field defined on the aneurysm surface. Two quantities are extracted from it: 

    - the aneurysm surface-averaged: 
\begin{equation}
    WSS_{av}
    =
    \frac{1}{A_a}
    \int_{S_a}{}
        \overline{WSS}(\vec{x})
    \mathrm{d}S_a   
\end{equation}

    - the maximum value of $\overline{WSS}(\vec{x})$:
\begin{equation}
    WSS_{max} 
    =
    \max_{\vec{x} \in S_a} \overline{WSS}(\vec{x})
\end{equation}

In [7]:
# We can work directly from the OSI surface with ContourScalars
# The time averaged values are defined per unit blood density
timeAveragedSurface = pv.XMLPolyDataReader(FileName=pathToFoamCase+"OSI_withAneurysm.vtp")

In [10]:
timeAveragedSurface.CellArrayStatus, timeAveragedSurface.PointArrayStatus

(['OSI', 'RRT', 'WSS_average', 'WSS_magnitude_average', 'wallShearComponent_average'],
 ['AneurysmNeckContour'])

In [ ]:
def wss(timeAveragedSurface, )

In [12]:
clipAneurysmNeck = pv.Clip()
clipAneurysmNeck.Input = timeAveragedSurface
clipAneurysmNeck.ClipType = 'Scalar'
clipAneurysmNeck.Scalars = [timeAveragedSurface.PointData.GetArray('AneurysmNeckContour').Name]
clipAneurysmNeck.Invert = 1   # gets smaller portion
clipAneurysmNeck.Value = 0.5  # based on the definition of field ContourScalars
clipAneurysmNeck.UpdatePipeline()

In [13]:
# Finaly we integrate over Sa 
integrateOverAneurysm = pv.IntegrateVariables()
integrateOverAneurysm.Input = clipAneurysmNeck
integrateOverAneurysm.UpdatePipeline()

In [16]:
aneurysmArea = integrateOverAneurysm.CellData.GetArray("Area").GetRange()[0] # aneurysm area
WSSav = integrateOverAneurysm.CellData.GetArray("WSS_magnitude_average").GetRange()[0]/aneurysmArea # averaged
WSSmax = clipAneurysmNeck.CellData.GetArray("WSS_magnitude_average").GetRange()[1] # maximum value

aneurysmArea, WSSav, WSSmax

(0.00018896121169792123, 270.9739053106728, 2678.042248705813)

# Post-process aneurysm surface integrals over time

The procedure below calculates the integral over the aneurysm surface $S_a$ of the WSS magnitude, which is a time-dependent integral:

\begin{equation}
    \left \langle \vec{WSS} \right \rangle (t)
    =
    \frac{1}{A_a}
    \int_{S_a}{}
        \left \| \vec{WSS} \right \|
    \mathrm{d}S_a   
\end{equation}

which is a measure of the total traction due to the tangential component of viscous traction on the aneurysm surface $S_a$. 

The procedure loads an OpenFOAM case of the aneurysm simulation and another surface must be loaded where is stored the array to clip the aneurysm neck: this can be performed in VMTK on a surface where the time averaged field were computed (time-average of WSS magnitude, magnitude of time-averaged WSS vector and OSI). 

In [ ]:
def area_averaged_wss_aneurysm(ofCaseFile,aneurysmClipSurface,aneurysmNeckArrayName,density=1056.0):
    """
        Function to compute surface integrals of 
        WSS over an aneurysm surface. It takes the 
        OpenFOAM case file and an extra surface where 
        it is stored a field with the aneurysm neck 
        line loaded as a ParaView PolyData surface.
    """
    # Read OpenFOAM data
    ofData = pv.OpenFOAMReader(FileName=ofCaseFile)

    # Update arrays to be used:
    # - wallShearComponent
    # - pressure
    ofData.CellArrays = ['wallShearComponent']

    # And select the surface where integration will be carried out
    ofData.MeshRegions = ['wall']

    # Get time-steps values
    timeSteps = np.array(ofData.TimestepValues)

    # Update time-step
    ofData.UpdatePipeline()

    # Merge blocks of input case
    mergeBlocks = pv.MergeBlocks()
    mergeBlocks.Input = ofData
    mergeBlocks.UpdatePipeline()
        
    areaAveragedWSSList = []
    # Iterate over time-steps to compute time dependent variables
    # only on the aneurysm surface: mag of WSS over time 
    for timeStep in timeSteps[-100:-1]:
        # Resample data: copy array of Input data (surface with time-averaged data) 
        # to Source data (the OpenFOAM simulated case)
        resampleDataset = pv.ResampleWithDataset()
        resampleDataset.Input = aneurysmClipSurface
        resampleDataset.Source = mergeBlocks
        resampleDataset.PassCellArrays = 1
        resampleDataset.PassPointArrays = 1
        resampleDataset.UpdatePipeline(time=timeStep)

        # Clipping aneurysm surface on its neck 
        clipAneurysmNeck = pv.Clip()
        clipAneurysmNeck.Input = resampleDataset
        clipAneurysmNeck.ClipType = 'Scalar'
        clipAneurysmNeck.Scalars = [aneurysmNeckArrayName]
        clipAneurysmNeck.Invert = 1   # gets smaller portion
        clipAneurysmNeck.Value = 0.5  # based on the definition of field ContourScalars
        clipAneurysmNeck.UpdatePipeline(time=timeStep)

        # Compute magnitude of WSS in each cell of the aneurysm surface
        magWSS = pv.Calculator()
        magWSS.Input = clipAneurysmNeck
        magWSS.ResultArrayName = 'magWSS'
        magWSS.Function = str(density)+'*mag('+clipAneurysmNeck.CellData.GetArray('wallShearComponent').Name+')'
        magWSS.AttributeType = 'Cell Data'
        magWSS.UpdatePipeline(time=timeStep)

        # Integrate WSS on the wall
        integrateWSS = pv.IntegrateVariables()
        integrateWSS.Input = magWSS
        integrateWSS.UpdatePipeline(time=timeStep)

        # Get area of surface, in m2
        aneurysmArea = integrateWSS.CellData.GetArray("Area").GetRange()[0]
        aneurysmArea

        # Instantiate calculater filter
        areaAveragedWSS = pv.Calculator()
        areaAveragedWSS.Input = integrateWSS
        areaAveragedWSS.ResultArrayName = 'areaAveragedWSS'
        areaAveragedWSS.Function = integrateWSS.CellData.GetArray('magWSS').Name+'/'+str(aneurysmArea)
        areaAveragedWSS.AttributeType = 'Cell Data'
        areaAveragedWSS.UpdatePipeline(time=timeStep)

        areaAveragedWSSList.append([timeStep, areaAveragedWSS.CellData.GetArray('areaAveragedWSS').GetRange()[0]])

    return np.asarray(areaAveragedWSSList)

In [ ]:
pathToFoamCase = "/home/iagolessa/foam/iagolessa-4.0/run/aneurysms/ruptured/fluidFlow/Casson/case2/mesh1500k/" 
foamFileName = "rupturedCase2Casson.foam"

In [ ]:
# Load surface with clip array and OSI data
# The surface with clip array must be constructed using VMTK,
# based on the surface with already 

clipArraySurface = pv.XMLPolyDataReader(FileName=pathToFoamCase+"OSI.vtp")
clipArraySurface.CellArrayStatus, clipArraySurface.PointArrayStatus

In [ ]:
type(clipArraySurface)

In [ ]:
aneurysmNeckArrayName = clipArraySurface.PointData.GetArray(0).Name
aneurysmNeckArrayName

In [ ]:
array = area_averaged_wss_aneurysm(pathToFoamCase+foamFileName, clipArraySurface, aneurysmNeckArrayName)

In [ ]:
rootPath = "/home/iagolessa/foam/iagolessa-4.0/run/aneurysms/"
models = ["Newtonian", "Casson", "CarreauYasuda"]
rupturedCases = ["case1", "case2", "case4", "case19"]
rupturedCasesFiles = [ rootPath+"ruptured/fluidFlow/"+model+"/"+case for model in models for case in rupturedCases ] 
rupturedCasesFiles

In [ ]:
a = np.array([5, 6, 7, 8])
df = pd.DataFrame({"a": [a]})
df

In [ ]:
timeSteps = np.array(ofData.TimestepValues)
timeSteps[-100:-1]

In [ ]:
array = np.zeros(shape=(4,3))
df = pd.DataFrame(array)
df.columns = ["Newt", "Casson", "CY"]
df.rename(index={0:"case1", 1:"case2", 2:"case4", 3:"case19"}, inplace=True)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sb
plt.style.use('classic')

In [ ]:
%matplotlib widget

fig = plt.figure()

plt.plot(array[:,0], array[:,1], 'b')

plt.xlabel('Time')
plt.ylabel('Area-averaged WSS (Pa)')

plt.grid()
plt.show()